In [4]:
import tensorflow as tf
from tensorflow import keras

import keras_tuner as kt

In [2]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [3]:
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [5]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    # Tune the number of units in the first Dense layer
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    # Tune the learning rate for the optimizer
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(10))
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

    return model

Tunners: `RandomSearch`, `Hyperband`, `BayesianOptimization` and `Sklearn`

In [6]:
tuner = kt.Hyperband(model_builder, objective='val_accuracy', max_epochs=10, factor=3, directory='my_dir', project_name='intro_to_kt')

In [7]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

## Search for the better hyperparameters

In [8]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

Trial 30 Complete [00h 00m 35s]
val_accuracy: 0.878333330154419

Best val_accuracy So Far: 0.8922500014305115
Total elapsed time: 00h 09m 19s
INFO:tensorflow:Oracle triggered exit


In [9]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [11]:
best_hps.get('units'), best_hps.get('learning_rate')

(224, 0.001)

## Search for the better epoch number

In [12]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

Epoch 1/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.5047 - accuracy: 0.8215 - val_loss: 0.4134 - val_accuracy: 0.8541
Epoch 2/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3753 - accuracy: 0.8640 - val_loss: 0.3632 - val_accuracy: 0.8719
Epoch 3/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3364 - accuracy: 0.8765 - val_loss: 0.3461 - val_accuracy: 0.8772
Epoch 4/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3091 - accuracy: 0.8871 - val_loss: 0.3765 - val_accuracy: 0.8610
Epoch 5/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2912 - accuracy: 0.8922 - val_loss: 0.3257 - val_accuracy: 0.8836
Epoch 6/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2767 - accuracy: 0.8977 - val_loss: 0.3244 - val_accuracy: 0.8839
Epoch 7/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2618 - accuracy: 0.9031 - val_loss: 0.3357 - val_accuracy:

In [13]:
# get best epoch amount
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1

In [14]:
best_epoch

38

## Train with the best configs (hyperparametrs and epochs)

In [15]:
hypermodel = tuner.hypermodel.build(best_hps)
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/38
1500/1500 [==============================] - 4s 2ms/step - loss: 0.5031 - accuracy: 0.8222 - val_loss: 0.4038 - val_accuracy: 0.8524
Epoch 2/38
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3753 - accuracy: 0.8646 - val_loss: 0.3795 - val_accuracy: 0.8623
Epoch 3/38
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3371 - accuracy: 0.8757 - val_loss: 0.4086 - val_accuracy: 0.8493
Epoch 4/38
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3115 - accuracy: 0.8842 - val_loss: 0.3374 - val_accuracy: 0.8796
Epoch 5/38
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2901 - accuracy: 0.8930 - val_loss: 0.3195 - val_accuracy: 0.8857
Epoch 6/38
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2760 - accuracy: 0.8963 - val_loss: 0.3154 - val_accuracy: 0.8877
Epoch 7/38
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2613 - accuracy: 0.9027 - val_loss: 0.3425 - val_accuracy:

## Results

In [16]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 0s 1ms/step - loss: 0.4833 - accuracy: 0.8854
[test loss, test accuracy]: [0.4832727611064911, 0.8853999972343445]
